In [21]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import warnings

In [22]:
warnings.filterwarnings('ignore')
pd.set_option('precision', 2)

In [23]:
# 读取原始数据
msf_df = pd.read_sas(r"C:\Users\86196\Desktop\资产定价复现\data\MSF_19252020.sas7bdat",encoding='latin-1')

数据清洗

In [24]:
t0=pd.to_datetime('1970-01-01')
t1=pd.to_datetime('2021-01-01')
needcol=['PERMNO','DATE','RET','SHROUT','PRC','ALTPRC','EXCHCD','DLSTCD']

# 选择所需时间段和所需列
imsf=msf_df[(msf_df.DATE>=t0)&(msf_df.DATE<=t1)][needcol].copy()

# 按日期和股票代码排序
imsf.sort_values(by=['DATE', 'PERMNO'], inplace=True)

# 去除重复数据
imsf.drop_duplicates(subset=['PERMNO','DATE'],inplace=True)

In [25]:
# 将时间都调整为每个月最后一天，方便后续处理
imsf['DATE']+=pd.tseries.offsets.MonthEnd(0)

# 丢弃缺失值
testcol=['RET','SHROUT','PRC','ALTPRC']
imsf.dropna(subset=testcol,inplace=True)

# 退市处理，DLSTCD为500、520、551-574、582、584，RET为-0.3，其余值RET为-1
bankrupt=list(range(551,575))+[500,520,582,584]
imsf[imsf.DLSTCD.notna()].RET=-1
imsf[imsf.DLSTCD.isin(bankrupt)].RET=-0.3

数据添加

In [26]:
# 添加MktCap列
imsf['MktCap']=imsf['ALTPRC'].abs()*imsf['SHROUT']/1000

In [27]:
# 添加MktCap_FF列
imsf['MktCap_FF']=np.nan
imsf.loc[imsf.DATE.dt.month==6, 'MktCap_FF'] = imsf.loc[imsf.DATE.dt.month==6, 'MktCap']
FF_group=imsf.groupby(['PERMNO', imsf['DATE'].dt.year - (imsf['DATE'].dt.month < 6)])

imsf['MktCap_FF']=FF_group['MktCap_FF'].transform('first')

In [28]:
# 引入CPI信息,来源于https://www.bls.gov/cpi/data.htm，已经处理过，成为以2020-12为基准CPI比例的时间序列
CPII=pd.read_csv(r"C:\Users\86196\Desktop\资产定价复现\data\CPII.csv")
CPII.columns=['date','CPII']
CPII['date']=pd.to_datetime(CPII['date'])
CPII=CPII.set_index('date')

# 添加CPI列
imsf['CPII'] = CPII.loc[imsf['DATE']]['CPII'].values
imsf['MktCap_CPI']=imsf['MktCap']*imsf['CPII']
imsf['MktCap_FF_CPI']=imsf['MktCap_FF']*imsf['CPII']

In [29]:
# 添加Size列
imsf['Size']=np.log(imsf['MktCap'])
imsf['Size_FF']=np.log(imsf['MktCap_FF'])
imsf['Size_CPI']=np.log(imsf['MktCap_CPI'])
imsf['Size_FF_CPI']=np.log(imsf['MktCap_FF_CPI'])

In [30]:
# 引入beta信息
beta_df = pd.read_sas(r"C:\Users\86196\Desktop\资产定价复现\data\Beta_12M_2020.sas7bdat",encoding='latin-1')
ib=beta_df[(beta_df.date>=t0)&(beta_df.date<=t1)].dropna(subset=['beta'])
ib['date']+=pd.tseries.offsets.MonthEnd(0)

# 合并imsf和ib
imsf=imsf.merge(ib,how='left',left_on=['PERMNO','DATE'],right_on=['permno','date'])

# 丢弃没有beta信息的数据
imsf.dropna(subset=['beta'],inplace=True)

In [31]:
# 丢弃不需要的列
dropcol=['SHROUT', 'PRC', 'ALTPRC', 'DLSTCD','CPII','permno', 'date', 'signal','fdate']
imsf.drop(columns=dropcol,inplace=True)

In [32]:
# 引入市场信息,来源于https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
mkt_df=pd.read_csv(r"C:\Users\86196\Desktop\资产定价复现\data\F-F_Research_Data_Factors.CSV",nrows=1161)
# 处理时序，并转为时间序列
mkt_df=mkt_df.rename(columns={'Unnamed: 0':'Date'})
mkt_df['Date']=pd.to_datetime(mkt_df['Date'],format='%Y%m') 
mkt_df['Date']=mkt_df['Date']+pd.tseries.offsets.MonthEnd(0)
mkt_df=mkt_df.set_index('Date')

In [33]:
# 添加ExRET列
imsf['RF']=mkt_df.loc[imsf.DATE].RF.values
imsf.RET*=100
imsf['ExRET']=imsf.RET-imsf.RF

In [34]:
# 添加ExRET_t+1列
asist=imsf[['PERMNO','DATE','ExRET']].copy()
asist['DATE']-=pd.tseries.offsets.MonthEnd(1)
asist.columns=['PERMNO','DATE','ExRET_t+1']
imsf=imsf.merge(asist,how='left',on=['PERMNO','DATE'])
imsf.dropna(subset=['ExRET_t+1'],inplace=True)

数据展示及保存

In [35]:
# 形状
imsf.shape

(2852978, 16)

In [36]:
# 列信息
imsf.columns

Index(['PERMNO', 'DATE', 'RET', 'EXCHCD', 'MktCap', 'MktCap_FF', 'MktCap_CPI',
       'MktCap_FF_CPI', 'Size', 'Size_FF', 'Size_CPI', 'Size_FF_CPI', 'beta',
       'RF', 'ExRET', 'ExRET_t+1'],
      dtype='object')

In [37]:
# 日期信息
period=imsf.DATE.unique()
period.sort()
print('时间样本取样数：',len(period),'时间跨度：',pd.to_datetime(period[0]).strftime('%Y-%m'),'至',pd.to_datetime(period[-1]).strftime('%Y-%m'))

时间样本取样数： 602 时间跨度： 1970-10 至 2020-11


In [38]:
# 保存数据
imsf.to_csv(r"C:\Users\86196\Desktop\资产定价复现\data\imsf.csv",index=False)